# Term Sheet Data Extraction with Python

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DscvryAI/structurify-sdk/blob/main/examples/notebooks/term-sheet.ipynb)

Extract structured data from **term sheet** documents using AI-powered extraction.

> Extract key terms from investment term sheets (VC, PE, debt financing)

## What This Notebook Does

This notebook demonstrates how to use [Structurify](https://structurify.ai) to automatically extract data from term sheet documents.

**Fields Extracted:**
- Deal Type
- Company Name
- Investor(s)
- Lead Investor
- Investment Amount
- Pre-Money Valuation
- Post-Money Valuation
- Price Per Share
- Security Type
- Liquidation Preference

**Industry:** Private Equity / Venture Capital  
**Try it online:** [https://structurify.ai/extract/term-sheet](https://structurify.ai/extract/term-sheet)

## Prerequisites

- A [Structurify](https://structurify.ai) account
- API key from your [dashboard](https://app.structurify.ai)
- Term Sheet document(s) to process (PDF, image, or scanned document)

## 1. Install the Structurify SDK

The [Structurify Python SDK](https://pypi.org/project/structurify/) provides easy access to the extraction API.

In [ ]:
!pip install structurify -q
print('Structurify SDK installed!')

## 2. Configure API Key

Get your API key from [app.structurify.ai](https://app.structurify.ai) and add it to Colab secrets (recommended) or paste it below.

In [ ]:
from structurify import Structurify

# Option 1: Use Colab secrets (recommended)
# Add your API key to Colab secrets with name 'STRUCTURIFY_API_KEY'
try:
    from google.colab import userdata
    api_key = userdata.get('STRUCTURIFY_API_KEY')
except:
    api_key = None

# Option 2: Paste directly (not recommended for shared notebooks)
if not api_key:
    api_key = 'sk_live_your_api_key'  # Replace with your key

client = Structurify(api_key=api_key)
print('Client initialized!')

## 3. Create a Term Sheet Extraction Project

Create a project using the `ptpl_term_sheet` template which is pre-configured to extract all relevant fields from term sheet documents.

In [ ]:
# Create project with Term Sheet template
project = client.projects.create(
    name='Term Sheet - Colab Demo',
    template_id='ptpl_term_sheet'
)

print(f'Created project: {project["name"]}')
print(f'Project ID: {project["id"]}')
print(f'Template: Term Sheet')

## 4. Upload Your Document

Upload your term sheet document. Supported formats: PDF, PNG, JPG, TIFF, DOCX, XLSX.

In [ ]:
# Option 1: Upload from your computer
from google.colab import files

print('Select your document to upload:')
uploaded = files.upload()

# Get the uploaded filename
filename = list(uploaded.keys())[0]
print(f'Uploaded: {filename}')

In [ ]:
# Upload to Structurify
doc = client.documents.upload(
    project_id=project['id'],
    file_path=filename
)

print(f'Document uploaded: {doc["name"]}')
print(f'Document ID: {doc["id"]}')

## 5. Run AI Extraction

Run the extraction to automatically identify and extract all term sheet fields using AI.

In [ ]:
# Start extraction
job = client.extraction.run(project_id=project['id'])
print(f'Extraction started! Job ID: {job["id"]}')

# Wait for completion
print('Processing...')
completed = client.extraction.wait_for_completion(job['id'])

print(f'\nExtraction complete!')
print(f'Status: {completed["status"]}')
print(f'Fields extracted: {completed.get("completedTasks", "N/A")}')

## 6. View Extracted Data

Export and view the extracted data as a DataFrame.

In [ ]:
import pandas as pd
from io import StringIO

# Export as CSV
export = client.exports.create(
    project_id=project['id'],
    format='csv'
)

# Download and display
csv_data = client.exports.download(export['export']['id'])
df = pd.read_csv(StringIO(csv_data))

print(f'Extracted {len(df.columns)} fields:')
df

## 7. Save Results

Download the extracted data as CSV or JSON.

In [ ]:
# Save as CSV
output_file = 'term-sheet_extracted.csv'
df.to_csv(output_file, index=False)

# Download to your computer
files.download(output_file)
print(f'Downloaded: {output_file}')

## 8. Batch Processing (Multiple Documents)

Process multiple term sheet documents at once.

In [ ]:
# Upload multiple files
print('Select multiple documents to upload:')
uploaded_batch = files.upload()

# Upload all to project
for filename in uploaded_batch.keys():
    doc = client.documents.upload(
        project_id=project['id'],
        file_path=filename
    )
    print(f'Uploaded: {filename}')

# Run extraction on all
job = client.extraction.run(project_id=project['id'])
completed = client.extraction.wait_for_completion(job['id'])
print(f'\nAll documents processed!')

## Next Steps

- **Web Interface:** Try the no-code version at [https://structurify.ai/extract/term-sheet](https://structurify.ai/extract/term-sheet)
- **Documentation:** [docs.structurify.ai](https://docs.structurify.ai)
- **API Reference:** [GitHub](https://github.com/DscvryAI/structurify-sdk)
- **More Templates:** [structurify.ai/extract](https://structurify.ai/extract)

---

*This notebook extracts data from term sheet documents using [Structurify](https://structurify.ai) - AI-powered document extraction.*